## run model on sample of OpenSearch tweets

In [ ]:
import pkg_resources

def is_installed(package):
    try:
        pkg_resources.get_distribution(package)
        return True
    except pkg_resources.DistributionNotFound:
        return False

print(is_installed('cudf'))

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
# import concurrent.futures
# from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoModel, AutoTokenizer
import torch, spacy,nltk,subprocess, json, requests,string,csv

model_name = "distilroberta-base"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# !python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
n_gram_range = (1, 2)
stop_words = "english"
embeddings=[]

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

import nltk, string, numpy as np
# nltk.download('punkt')

In [ ]:
def get_npr_news(p):
    # Send a GET request to the NPR API
    r = requests.get("http://api.npr.org/query?apiKey=npr_key", params=p)

    # Parse the XML response to get the story URLs
    root = ET.fromstring(r.content)
    story_urls = [story.find('link').text for story in root.iter('story')]

    # For each story URL, send a GET request to get the HTML content
    full_stories = []
    for url in story_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content of the story. This will depend on the structure of the webpage.
        # Here, we're assuming that the main content is in a <p> tag. You might need to adjust this depending on the webpage structure.
        story = soup.find_all('p')

        # Extract the text from the story
        full_story = ' '.join(p.text for p in story)
        full_stories.append(full_story)
    return full_stories


In [ ]:
def chunk_text(text, max_len):
    # Tokenize the text into tokens
    tokens = nltk.word_tokenize(text)

    # Calculate the number of chunks and the size of the final chunk
    num_chunks = len(tokens) // max_len
    final_chunk_size = len(tokens) % max_len

    # If the final chunk is too small, distribute its tokens among the other chunks
    if final_chunk_size < max_len / 2:
        num_chunks += 1
        chunk_sizes = [len(tokens) // num_chunks + (1 if i < len(tokens) % num_chunks else 0) for i in range(num_chunks)]
        chunks = [tokens[sum(chunk_sizes[:i]):sum(chunk_sizes[:i+1])] for i in range(num_chunks)]
    else:
        chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len)]

    return chunks


In [ ]:
# from torch.utils.data import DataLoader
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoModel, AutoTokenizer
import torch

def featurize_stories(text, max_len, top_k):
    # Extract candidate words/phrases
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text])
    all_candidates = count.get_feature_names_out()
    doc = nlp(text)
    noun_phrases = set(chunk.text.strip().lower() for chunk in doc.noun_chunks)
    nouns = set()
    for token in doc:
        if token.pos_ == "NOUN":
            nouns.add(token.text)

    all_nouns = nouns.union(noun_phrases)
    candidates = list(filter(lambda candidate: candidate in all_nouns, all_candidates))
    candidate_tokens = tokenizer(candidates, padding=True, return_tensors="pt")
    # candidate_tokens = {k: v.to(device) for k, v in (candidate_tokens).items()}
    candidate_embeddings = model(**candidate_tokens)["pooler_output"]
    candidate_embeddings = candidate_embeddings.detach()#.to_numpy()

    # words = nltk.word_tokenize(text)
    # chunks = [words[i:i + 512] for i in range(0, len(words), 512)]
    chunks = chunk_text(text, max_len)  # use this to chunk better and use less padding thus less memory but also less affect from averging

    for chunk in chunks:
        text_tokens = tokenizer(chunk, padding=True, return_tensors="pt")
        text_tokens = {k: v.to(device) for k, v in (text_tokens).items()}
        text_embedding = model(**text_tokens)["pooler_output"]
        text_embedding = text_embedding.detach()#.to_numpy()
        embeddings.append(text_embedding)
    max_emb_shape = max(embedding.shape[0] for embedding in embeddings)
    padded_embeddings = [np.pad(embedding.cpu(), ((0, max_emb_shape - embedding.shape[0]), (0, 0))) for embedding in embeddings]
    avg_embedding = np.min(padded_embeddings, axis=0)
    distances = cosine_similarity(avg_embedding, candidate_embeddings.cpu())
    torch.cuda.empty_cache()
    return [candidates[index] for index in distances.argsort()[0][::-1][-top_k:]]



In [ ]:
import pandas as pd
# data=pd.read_csv('/content/drive/MyDrive/consult/Louie_disaster_tweets.csv',header=None)
data=pd.read_csv('florida-hurricane-tweet.csv')

In [ ]:
data.to_parquet('8.5k_florida_tweets.parquet')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features.txt',sep='\t')

In [ ]:
rank_articles=[]
from tqdm import tqdm
for i in tqdm(range(len(df),len(data['text']))):
    try:
        cc=featurize_stories(data['text'][i], max_len=512, top_k=4)
        # print(cc)
        rank_articles.append(cc)
    except IndexError:
        pass

In [ ]:
flattened_list = [item for sublist in rank_articles for item in sublist]
from collections import Counter
counter = Counter(flattened_list)
df = pd.DataFrame.from_dict(counter, orient='index', columns=['Count'])

df = df.sort_values(by='Count',ascending=False)
df.to_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features2.txt',sep='\t')
print(len(df))
# df[:25]

In [ ]:
filtered_list = [sublist for sublist in rank_articles if any('fire'.lower() in s.lower() for s in sublist)]
filtered_list

In [ ]:
import pandas as pd
df = pd.read_csv('florida-hurricane-tweet_features.txt',sep='\t')
df2=pd.read_csv('florida-hurricane-tweet_features2.txt',sep='\t')
df=pd.concat([df,df2])
print(df['Unnamed: 0'])
df = df.groupby('Unnamed: 0').sum().sort_values(by='Count',ascending=False)
df=df[df['Count']>int(np.round(len(df)*.001))]

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

# nouns = ['apple', 'John', 'London', 'dog', 'Mary', 'Paris', 'banana']
nouns= df.reset_index()['Unnamed: 0'].to_list()
doc = nlp(' '.join(nouns))

proper_nouns = [token.text for token in doc if token.pos_ == 'PROPN']

print(proper_nouns)

In [ ]:
print(len(proper_nouns))
proper_nouns

In [ ]:
len(nouns)

In [ ]:
i=['Iceland', '16-01-2024', ['Lava from a volcanic eruption in southwestern Iceland has streamed into a nearby town, engulfing homes and forcing the evacuation of local residents.\n', 'The Icelandic Meteorological Office said the eruption began just before 8:00 a.m. local time Sunday about a half mile from the town of Grindavík after a series of intense earthquakes.\n', 'A second fissure opened after noon Sunday and sent lava flows into the town, officials said.\n', 'Iceland\'s president, Guðni Th. Jóhannesson, said in an address to the nation on Sunday that a "daunting period of upheaval" had begun for those in the Reykjanes peninsula.\n', '"We continue to hope for as good an outcome as possible, in the face of these tremendous forces of nature," he said.\n', 'This is the second time in a month that a volcano has erupted just outside Grindavík, a coastal town about 25 miles from the Icelandic capital of Reykjavík.\n', 'An eruption on December 18 sent lava spewing into the air, but residents had already been told to leave due to heightened seismic activity.\n', "Iceland's government said Sunday that Grindavík had been evacuated early that morning and that the eruption isn't expected to reach other populated areas.\n", 'So far no flights have been disrupted, and officials are monitoring threats to infrastructure. At least three homes have either burned down or been overtaken by lava, according to the Icelandic broadcaster RUV.\n', "The government also said the eruption doesn't present a threat to life.\n", 'But in his speech, Jóhannesson offered his sympathy to the loved ones of Lúðvík Pétursson, a man who went missing in a work accident in Grindavík last week.\n', "According to Sky News, the 50-year-old Pétursson was filling crevasses formed by volcanic activity and earthquakes when he fell in a crack that had opened after last month's eruption.\n", 'Iceland is a hotspot for seismic activity, with 32 active volcanoes. A volcano erupts roughly every five years in the country, though eruptions have occurred more frequently recently. \n', ' Copyright 2024 NPR. To see more, visit https://www.npr.org.  ', '\n'], None, None, None]

## parameterize functions
 - parallelize text pulls from urls

In [ ]:
import argparse, signal,datetime,subprocess,json,os
from dotenv import load_dotenv
parser = argparse.ArgumentParser(description='Process OS data for dynamic features.')
parser.add_argument('-n', type=int, default=10, help='Number of data items to get')
parser.add_argument('-f', type=int, default=3, help='Number of features per item to get')
parser.add_argument('-o', type=str, default='dots_feats.csv', help='Output file name')
parser.add_argument('-p', type=int, default=1, help='Parallelize requests')
# parser.add_argument('-s', type=datetime, default=20230101, help='start date')
# parser.add_argument('-e', type=datetime, default=20231231, help='end date')
args, unknown = parser.parse_known_args()
load_dotenv()

In [ ]:
from dots_feat import get_OS_data
dd=get_OS_data(10)
dd

In [ ]:
# os_url=([os.environ['OS_TOKEN']])

os_url = os.getenv('OS_TOKEN')
n=10
bash_command = f"""
curl -X GET "{os_url}/emergency-management-news/_search?scroll=1m" -H 'Content-Type: application/json' -d '{{
"_source": ["metadata.GDELT_DATE", "metadata.page_title","metadata.DocumentIdentifier", "metadata.Organizations","metadata.Persons","metadata.Themes","metadata.text", "metadata.Locations"],
    "size": {n},
    "query": {{
        "bool": {{
            "must": [
                {{"match_all": {{}}}}
            ]
        }}
    }}
}}'
"""
process = subprocess.run(bash_command, shell=True, capture_output=True, text=True)
output = process.stdout
data = json.loads(output)
data

In [ ]:
output

In [ ]:
# data = json.loads("input/feat_input.json")
with open("input/feat_input.json", 'r') as f:
    data = json.load(f)
data['_scroll_id']

In [ ]:
len(data['hits']['hits'])

In [ ]:
# Keep track of the scroll ID
scroll_id = response['_scroll_id']
es = OpenSearch([os.environ['OS_TOKEN']])
# Keep scrolling until no more results
while len(response['hits']['hits']):
    # print(response['hits']['hits'])  # process results

    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # length of time to keep the scroll window open
    )

    # Update the scroll ID
    scroll_id = response['_scroll_id']


In [ ]:
from dots_feat import featurize_stories, process_data, get_OS_data, process_hit
data = get_OS_data(10)
data['hits']['hits']

In [ ]:
import sys,os,argparse,datetime
sys.argv = ['dots_feat.py', '-n', '10', '-f', '3', '-o', 'dots_feats.csv', '-p', '1']#, '-s', '20230101', '-e', '20231231']
parser = argparse.ArgumentParser(description='Process OS data for dynamic features.')
parser.add_argument('-n', type=int, default=10, help='Number of data items to get')
parser.add_argument('-f', type=int, default=3, help='Number of features per item to get')
parser.add_argument('-o', type=str, default='dots_feats.csv', help='Output file name')
parser.add_argument('-p', type=int, default=1, help='Parallelize requests')
# parser.add_argument('-s', type=datetime.date, default=datetime.datetime.strptime(20230101), help='start date')
# parser.add_argument('-e', type=datetime.date, default=datetime.datetime.strptime(20231231), help='end date')
args, unknown = parser.parse_known_args()

from dots.dots_feat import featurize_stories, process_data, get_OS_data, process_hit

In [ ]:
import dots

In [ ]:
s="20230101"
e="20231231"

s_date = datetime.datetime.strptime(s, "%Y%m%d")
e_date = datetime.datetime.strptime(e, "%Y%m%d")

# Convert datetime objects to ISO 8601 format
s_iso = s_date.isoformat() + "Z"
e_iso = e_date.isoformat() + "Z"

In [ ]:
[s_iso,e_iso]

In [ ]:
def get_OS_data(n=args.n):
    bash_command = f"""
    curl -X GET "{os_url}" -H 'Content-Type: application/json' -d '{{
"_source": ["metadata.GDELT_DATE", "metadata.page_title","metadata.DocumentIdentifier", "metadata.Organizations","metadata.Persons","metadata.Themes","metadata.text", "metadata.Locations"],
        "size": {n},
        "query": {{
            "bool": {{
                "must": [
                    {{"match_all": {{}}}},
                ]
            }}
        }}
    }}'
    """
    process = subprocess.run(bash_command, shell=True, capture_output=True, text=True)
    output = process.stdout
    data = json.loads(output)
    return data

In [ ]:
data = get_OS_data(10)
data

In [ ]:
data = get_OS_data(10)

In [ ]:
data = get_OS_data(10)
hits = data['hits']['hits']
hit = hits[1]
hit['_source']

from datetime import datetime
source = hit['_source']
date = datetime.strptime(source['metadata']['GDELT_DATE'], "%Y%m%d%H%M%S")
loc = source['metadata']['Locations']
# ex = None #source['metadata']['Extras']
title = source['metadata']['page_title']
url = source['metadata']['DocumentIdentifier']

In [ ]:
s = source['metadata']['Locations']

s = s.replace("'", '"')  # json requires double quotes for keys and string values
list_of_dicts = json.loads(s)


# location_full_name = source['metadata']['Locations']['Location FullName']

for dict in list_of_dicts:
    if 'Location FullName' in dict:
        print(dict['Location FullName'])

In [ ]:
data

In [ ]:

def process_data_fast(data):
    articles = []
    hits = data['hits']['hits']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_hit, hits))

    with open('output.csv', 'w') as file:
        writer = csv.writer(file)
        for text, soup,date,loc,title in results:
            articles.append([loc,loc,loc,date,title,title,text])
            # articles.append(text)

            writer.writerow([loc,loc,loc,date,title,title,text]) # force location into top feature, also assume title has important info
            # writer.writerow(text)
            writer.writerow(['\n'])

    with open('output.html', 'w') as file:
        file.write(str(results[-1][1]))  # Write the soup of the last hit

    # with open('output.csv', 'w') as file:
    #     file.write(str(results[-1][0]))  # Write the z of the last hit

    return articles


In [ ]:
from dots_feat import process_data_fast

In [ ]:
def process_hits(hit):
    source = hit['_source']
    GDELT_DATE = source['metadata']['GDELT_DATE']
    page_title = source['metadata']['page_title']
    url = source['metadata']['DocumentIdentifier']
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to get {url}")
        return z
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all(['p','div','a'])
    if not paragraphs:
        print(f"No <p> tags in {url}")
        return z

    for p in paragraphs:
        z.append(p.get_text())
    return z

with open('output.html', 'w') as file:
    file.write(str(soup))
    
with open('output.csv', 'w') as file:
    file.write(str(z))

# return articles
articles=[]
hits = data['hits']['hits']
for hit in hits:
    z=process_hits(hit)
    articles.append(z)
    

In [ ]:

def process_hit(hit):
    z = []
    source = hit['_source']
    GDELT_DATE = source['metadata']['GDELT_DATE']
    page_title = source['metadata']['page_title']
    location = source['metadata']["Locations"]
    extras = source['metadata']["Extras"]
    url = source['metadata']['DocumentIdentifier']

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for p in soup.find_all('p'):
        z.append(p.get_text())
    z.append(page_title)
    return z, soup

def process_data_fast(data):
    articles = []
    hits = data['hits']['hits']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_hit, hits))

    with open('output.csv', 'w') as file:
        writer = csv.writer(file)
        for z, soup in results:
            writer.writerow(z)
            writer.writerow(['\n'])
            articles.append(z)

    with open('output.html', 'w') as file:
        file.write(str(results[-1][1]))  # Write the soup of the last hit

    # with open('output.csv', 'w') as file:
    #     file.write(str(results[-1][0]))  # Write the z of the last hit

    return articles


In [ ]:

data['hits']['hits']

In [ ]:
data = get_OS_data(10)
# articles = process_data(data)
hits = data['hits']['hits']

In [ ]:
import concurrent.futures
results=[]
e = concurrent.futures.ThreadPoolExecutor()
    # try:
future = e.submit(process_hit, hits[1])
result = future.result(timeout=5)  # Set timeout for 5 seconds
results.append(result)

In [ ]:
data['hits']['hits'][1]['_source']

## process hits

In [ ]:
from datetime import datetime
import requests,os,csv,logging
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from opensearchpy import OpenSearch
load_dotenv()

In [ ]:
def process_response(response):
    hits = response["hits"]["hits"]
    output=[]
    for iii,hit in enumerate(hits):

        source = hit["_source"]
        text=[]
        try:
            date = datetime.strptime(source['metadata']['GDELT_DATE'], "%Y%m%d%H%M%S")
            date = formatted_date = date.strftime("%d-%m-%Y")
            loc = source['metadata']['Locations']
            loc = loc.replace("'", '"')  # json requires double quotes for keys and string values
            try:
                list_of_dicts = json.loads(loc)
                location_full_names = [dict['Location FullName'] for dict in list_of_dicts if 'Location FullName' in dict]
                loc = location_full_names[0]
            except:
                loc = None
            org = source['metadata']['Organizations']
            per = source['metadata']['Persons']
            theme = source['metadata']['Themes'].rsplit('_')[-1]
            title = source['metadata']['page_title']
            url = source['metadata']['DocumentIdentifier']
            output.append([date, loc, title, org, per, theme, url])
            # pagination_id=response['_scroll_id']
        
        except:
            # pagination_id=None
            pass
    return output
    

In [ ]:
response, client = get_massive_OS_data(args.n,args.t)

In [ ]:
# response['hits']['hits']
# response["_scroll_id"]

In [ ]:
articles=[]
hits = response["hits"]["hits"]
while len(hits) != 0:
    # try:
    response = client.scroll(
        scroll='5m',
        scroll_id=pagination_id
            )
    hits = response["hits"]["hits"]
    article = process_response(response)
    articles.append(article)
    # except:
    #     print("A ConnectionTimeout error occurred.")
    #     pass
    

In [ ]:
len(articles)

In [ ]:

flattened_list = [item for sublist in articles for item in sublist]

## scroll OS

In [ ]:
# os.getcwd()
# os.chdir('..')

In [ ]:
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from scrape import get_OS_data, get_massive_OS_data,get_google_news
from pull import process_hit, process_data,pull_data
os.getcwd()
# os.chdir("..")

In [ ]:
def get_scroll_data():
    response, client = get_massive_OS_data(1)
    pagination_id = response["_scroll_id"]
    hits = response["hits"]["hits"]
    articles=[]
    articles2=[]
    while len(hits) != 0 and len(articles2) < 11000:
        response = client.scroll(
            scroll='1m',
            scroll_id=pagination_id
                )
        hits = response["hits"]["hits"]
        # article = process_data(response)
        articles.append(hits)
        articles2 = [item for sublist in articles for item in sublist]
    return [item for sublist in articles for item in sublist]

In [ ]:
def extract_location(location_str):
    if location_str:
        try:
            location_list = json.loads(location_str.replace("'", '"'))
            return [dict['Location FullName'] for dict in location_list if 'Location FullName' in dict]
        except json.JSONDecodeError:
            return None
    else:
        return None

def process_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = []
    for p in paragraphs:
        text.append(p.get_text())
    return text

import json
import concurrent.futures, requests
from bs4 import BeautifulSoup

In [ ]:
data = [list(d['_source']['metadata'].values()) for d in articles]
df = pd.DataFrame(data, columns=['date','title', 'person', 'org', 'location', 'theme', 'text', 'url'])
df.date=pd.to_datetime(df.date).dt.strftime('%d-%m-%Y')
df['locc'] = df['location'].apply(extract_location)
with concurrent.futures.ThreadPoolExecutor() as executor:
    df['text'] = list(executor.map(process_url, df['url']))
return df.values.tolist()

## small gnews 

In [ ]:
import os
os.getcwd()
# os.chdir('..')

In [ ]:
from gnews import GNews
from tqdm import tqdm
from scrape import get_OS_data, get_massive_OS_data,get_google_news
from pull import process_hit, process_data,pull_data
# from feat import featurize_stories
google_news = GNews()

# google_news.period = '7d'  # News from last 7 days
google_news.max_results = 10000  # number of responses across a keyword
google_news.country = 'United States'  # News from a specific country 
google_news.language = 'english'  # News in a specific language
google_news.exclude_websites = ['yahoo.com', 'cnn.com']  # Exclude news from specific website i.e Yahoo.com and CNN.com
google_news.start_date = (2024, 1, 1) # Search from 1st Jan 2020
google_news.end_date = (2024, 3, 1) # Search until 1st March 2020



In [ ]:
json_resp = google_news.get_news('disaster')
article=[]

for i in tqdm(range(len(json_resp)), desc="grabbing directly from GoogleNews"):
    aa=(google_news.get_full_article(json_resp[i]['url']))
    try:
        date=aa.publish_date.strftime("%d-%m-%Y")
    except:
        date=None
    try:
        title=aa.title
        text=aa.text
    except:
        title=None
        text=None
    article.append([title,date,text])

# return article

In [ ]:
article

In [ ]:
# farticles = [item for sublist in article for item in sublist]
rank_articles=[]
for i in tqdm(article, desc="featurizing articles"):
    foreparts=str(i).split(',')[:2]  # location and date
    meat="".join(str(i).split(',')[2:-3])  # article text
    cc=featurize_stories(str(i), top_k = 3, max_len=512)
    rank_articles.append([foreparts,cc])

## lobstr API

In [ ]:
import os,json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
lobstr_key = os.environ['LOBSTR_KEY']

In [ ]:
AA['id'][0]

In [ ]:
!curl 'https://api.lobstr.io/v1/runs/{AA['id'][0]}/stats' \
    -H 'Accept: application/json' \
    -H "Authorization: Token $lobstr_key" \

In [ ]:

!curl 'https://api.lobstr.io/v1/runs?page=1&page_size=3000' \
    -H 'Accept: application/json' \
    -H "Authorization: Token $lobstr_key" \
    -o 'input/runs.json'
    
with open("input/runs.json", 'r') as f:
    runs = json.load(f)
juns=pd.DataFrame(runs['data'])
AA=juns[['id','cluster','total_unique_results']]
latest_success_run = AA.loc[AA['total_unique_results'].ne(0).idxmax()]

!curl 'https://api.lobstr.io/v1/results?cluster=8de6e1bbf33f47b8bce451075b883252&run={latest_success_run['id']}&page=1&page_size=3000' \
    -H 'Accept: application/json' \
    -H "Authorization: Token $lobstr_key" \
    -o 'input/lobstr_results.json'
    
with open("input/lobstr_results.json", 'r') as f:
    data = json.load(f)
    
jata=pd.DataFrame(data['data'])
jata[['published_at','url','title','short_description']]

In [ ]:
with open("input/lobstr_results.json", 'r') as f:
    data = json.load(f)
    
jata=pd.DataFrame(data['data'])
jata[['published_at','url','title','short_description']]

## lobstr via gdocs

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/178sqEWzqubH0znhx7Z6u9ig2EjCRvl0dUsA7b6hQpmY/export?format=csv'
import pandas as pd
df = pd.read_csv(url)
df=df[['published_at','url','title','short_description']]

In [ ]:
articles = pd.read_csv('input/lobstr_text.txt', sep='\t', sep='\t', index_col=0, names=['text'], skiprows=1)
url = 'https://docs.google.com/spreadsheets/d/178sqEWzqubH0znhx7Z6u9ig2EjCRvl0dUsA7b6hQpmY/export?format=csv'
df = pd.read_csv(url)
len(articles) == len(df)
# len(articles)

In [ ]:
articles.to_parquet('input/lobstr_text.parquet', index=False)#, mode='a')

In [ ]:
jj=pd.read_parquet('input/lobstr_text.parquet')

In [ ]:
jj

In [ ]:
import torch
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == 'cuda':
    dataloader = DataLoader(data['text'], batch_size=1, shuffle=True, num_workers=4)
    RR = dataloader
else:
    RR = articles
for j,i in tqdm(enumerate(RR), total=len(RR), desc="featurizing articles"):

# for i in tqdm(articles, desc="featurizing articles"):
    try:
        foreparts = str(i).split(',')[:2]  # location and date
    except:
        foreparts=None
    meat="".join(str(j).split(',')[2:-3])  # text
    try:
        cc=featurize_stories(str(i), top_k = args.f, max_len=512)
    except:
        'damn'

In [ ]:
RR

In [ ]:
# from multiprocessing import Pool
from tqdm import tqdm
import requests, concurrent
from bs4 import BeautifulSoup
# def handler(signum, frame):
    # raise TimeoutError()/
# signal.signal(signal.SIGALRM, handler)


def process_url(url):
    try:
        # signal.alarm(5)
        response = requests.get(url,timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all(['p'])
        text = []
        for p in paragraphs:
            text.append(p.get_text())
        # signal.alarm(0)
        return text
    except Exception as e:
        # print(f"Error processing URL {url}: {e}")
        return None
    

In [ ]:
# results=[]
# for i in tqdm(df['url'], desc="grabbing text from url"):
#     result = process_url(i)
#     results.append(result)
# df['text'] = results

In [ ]:
results=[]
with concurrent.futures.ThreadPoolExecutor() as executor:
    result = list(tqdm(executor.map(process_url, df['url']), total=len(df['url'])))

In [ ]:
# farticles = [item for sublist in article for item in sublist]
rank_articles=[]
for i in tqdm(result, desc="featurizing articles"):
    foreparts = df[['published_at','title']]
    try:
        cc=featurize_stories(str(i), top_k = 3, max_len=512)
    except:
        cc=None
        pass
    rank_articles.append([foreparts,cc])

In [ ]:
pd.DataFrame(rank_articles)

In [ ]:
# farticles = [item for sublist in article for item in sublist]
rank_articles=[]
for i in tqdm(df['text'], desc="featurizing articles"):
    foreparts = df['datepublished_at','title'].tolist()
    cc=featurize_stories(str(i), top_k = 3, max_len=512)
    rank_articles.append([foreparts,cc])

In [12]:
hits[1]['_index']['metadata']

TypeError: string indices must be integers, not 'str'

## test-google-news-index

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/apple/WRK/dcolinmorgan/dots'

In [4]:
from DOTS.scrape import get_OS_data, get_test_gnews
from DOTS.pull import pull_data, process_url
from DOTS.ingestion_utils import safe_iter_pull, iter_pull, reduce_newlines, scrape_selenium_headless

In [5]:
from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
os_url = os.getenv('OS_TOKEN')
lobstr_key = os.getenv('LOBSTR_KEY')

In [29]:
# response = get_test_gnews(100)
# hits = response["hits"]["hits"]
# articles = pull_data(hits)
# indices = [i for i, x in enumerate(articles) if len(str(x)) < 50]
print(len(indices))
for i in indices:
    url = hits[i]['_source']['metadata']['link']
    print(url)
    try:
        articles[i] = scrape_selenium_headless(url,browser='undetected_chrome')
    except:
        pass
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
print(len(indices2))

22
https://news.google.com/rss/articles/CBMihgFodHRwczovL3d3dy5hbmluZXdzLmluL25ld3Mvd29ybGQvbWlkZGxlLWVhc3QvODE0LWlzcmFlbGktY2l2aWxpYW5zLWtpbGxlZC1pbi1wYWxlc3Rpbmlhbi10ZXJyb3ItYXR0YWNrcy1zaW5jZS1vY3RvYmVyLTcyMDI0MDQwMjIxNDc1M9IBAA?oc=5
https://news.google.com/rss/articles/CBMi5AFodHRwOi8vd3d3Lm1zbi5jb20vZW4teGwvYWZyaWNhL3RvcC1zdG9yaWVzL21vc2Nvdy1hdHRhY2stcnVzc2lhLWFycmVzdHMtbW9yZS1hdHRhY2stcGxvdHRlcnMvYXItQkIxa1NYVko_b2NpZD1YTU1PJmFwaXZlcnNpb249djImbm9zZXJ2ZXJjYWNoZT0xJmRvbXNoaW09MSZyZW5kZXJ3ZWJjb21wb25lbnRzPTEmd2NzZW89MSZiYXRjaHNlcnZlcnRlbGVtZXRyeT0xJm5vc2VydmVydGVsZW1ldHJ5PTHSAQA?oc=5
https://news.google.com/rss/articles/CBMiX2h0dHBzOi8vd3d3Lm1zbi5jb20vZW4temEvbmV3cy93b3JsZC93aHktdGhlLXVzLXdhcm5lZC1ydXNzaWEtb2YtYS10ZXJyb3Jpc3QtYXR0YWNrL2FyLUJCMWtWRXh50gEA?oc=5
https://news.google.com/rss/articles/CBMibmh0dHBzOi8vd3d3LnJldXRlcnMuY29tL3dvcmxkL2V1cm9wZS9ydXNzaWEtc2F5cy1pdC1pcy13b3JraW5nLXJlbW92aW5nLXRhbGliYW4taXRzLXRlcnJvcmlzdC1saXN0LTIwMjQtMDQtMDIv0gEA?oc=5
https://news.google.com/rss/

In [27]:
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
print(len(indices2))

14


In [ ]:
user_agent_list = [ 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36', 'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1', 'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363', ]
scrapy.Request(url, callback=self.parse, headers={ "User-Agent": self.user_agent_list[random.randint(0, len(self.user_agent_list) - 1)]})

In [ ]:
# import os
# os.chdir('..')
os.getcwd()

In [ ]:
import sys
sys.path.append('/Users/apple/WRK/dcolinmorgan/dots/DOTS/msn_scraper/')


from scrapy.crawler import CrawlerProcess
from msn_scraper.spiders.msn_spider import msn_spider

# process = CrawlerProcess({
#     'USER_AGENT': 'Mozilla/5.0 (compatible; MSIE 7.0; Windows NT 5.1)'
# })

process.crawl(msn_spider)
process.start()  # the script will block here until the crawling is finished

In [ ]:
import random, scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from bs4 import BeautifulSoup

class msn_spider(scrapy.Spider):
    name = "msn"

    start_urls = [
        'https://news.google.com/rss/articles/CBMiZGh0dHBzOi8vcGVyc3BlY3RpdmVzbWVkLmNvbS91bi1pc3JhZWxpZW4tYXJyZXRlLWEta3VhbGEtbHVtcHVyLXZ1bGdhaXJlLWNyaW1pbmVsLW91LWFnZW50LWR1LW1vc3NhZC_SAQA?oc=5',
    ]
    user_agent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
        'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
        'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363', 
    ]

    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url, headers={'User-Agent': random.choice(self.user_agent_list)})

    def parse(self, response):
        html = response.text
        # for paragraph in response.css('p'):
            # yield {'text': paragraph.get()}
        # soup = BeautifulSoup(html, 'html.parser')
        # see = reduce_newlines(soup.get_text())
        with open('output.txt', 'a') as f:
            f.write(html)

process = CrawlerProcess(get_project_settings())
process.crawl(msn_spider)
process.start()  # the script will block here until the crawling is finished

2024-04-08 16:40:50 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-04-08 16:40:50 [scrapy.utils.log] INFO: Versions: lxml 5.1.0.0, libxml2 2.12.3, cssselect 1.2.0, parsel 1.9.0, w3lib 2.1.2, Twisted 24.3.0, Python 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:54:21) [Clang 16.0.6 ], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform macOS-14.4.1-arm64-arm-64bit
2024-04-08 16:40:50 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-08 16:40:50 [py.warnings] WARNING: /opt/homebrew/Caskroom/mambaforge/base/envs/DT/lib/python3.12/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of S

In [24]:
import json
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

capabilities = DesiredCapabilities().CHROME

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--ignore-certificate-errors")

# disable the banner "Chrome is being controlled by automated test software"
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

# driver = webdriver.Chrome('/usr/local/bin/chromedriver', options=chrome_options)
webdriver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
url_main = 'https://www.scmp.com/news/asia/east-asia/article/3199400/japan-asean-hold-summit-tokyo-around-december-2023-japanese-official'
# url_main = 'https://news.google.com/rss/articles/CBMiZGh0dHBzOi8vcGVyc3BlY3RpdmVzbWVkLmNvbS91bi1pc3JhZWxpZW4tYXJyZXRlLWEta3VhbGEtbHVtcHVyLXZ1bGdhaXJlLWNyaW1pbmVsLW91LWFnZW50LWR1LW1vc3NhZC_SAQA?oc=5'

driver.get(url_main)

driver.implicitly_wait(20)
element_has_bottom_message = WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.CLASS_NAME, "has-bottom-messaging")))
if element_has_bottom_message:
    element_gdpr = WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.CLASS_NAME, "gdpr-banner__accept")))
    if element_gdpr:
        gdrp_button = driver.find_element_by_xpath("//*[@class='gdpr-banner__accept']")
        driver.implicitly_wait(20)
        ActionChains(driver).move_to_element(gdrp_button).click(gdrp_button).perform()
        element_my_news_popup = WebDriverWait(driver, 120).until(
            EC.presence_of_element_located((By.CLASS_NAME, "my-news-landing-popup__icon-close")))
        if element_my_news_popup:
            my_news_popup = driver.find_element_by_xpath("//*[@class='my-news-landing-popup__icon-close']")
            ActionChains(driver).move_to_element(my_news_popup).click(my_news_popup).perform()
            driver.implicitly_wait(20)
            raw_soup = BeautifulSoup(driver.page_source, 'lxml')
            json_dictionaries = raw_soup.find_all(name='script', attrs={'type': 'application/ld+json'})
            if len(json_dictionaries) != 0:
                for json_dictionary in json_dictionaries:
                    dictionary = json.loads("".join(json_dictionary.contents), strict=False)
                    article_bool = bool([value for (key, value) in dictionary.items() if key == 'articleBody'])
                    if article_bool:
                        for key, value in dictionary.items():
                            if key == 'articleBody':
                                print(value)


sleep(30)
driver.close()
driver.quit()

WebDriverException: Message: disconnected: unable to send message to renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=123.0.6312.107)
Stacktrace:
0   chromedriver                        0x0000000102950474 chromedriver + 4326516
1   chromedriver                        0x000000010294893c chromedriver + 4294972
2   chromedriver                        0x0000000102574088 chromedriver + 278664
3   chromedriver                        0x000000010255d468 chromedriver + 185448
4   chromedriver                        0x000000010255bc5c chromedriver + 179292
5   chromedriver                        0x000000010255c930 chromedriver + 182576
6   chromedriver                        0x0000000102577078 chromedriver + 290936
7   chromedriver                        0x00000001025778c0 chromedriver + 293056
8   chromedriver                        0x000000010257a428 chromedriver + 304168
9   chromedriver                        0x000000010257a4a

In [ ]:
# Create an instance of the spider
spider = msn_spider()

# Call the start_requests function and store the returned list in a variable
requests = spider.start_requests()

# Print the requests
for request in requests:
    print(request)

In [ ]:
from DOTS.feat import featurize_stories
import graphistry
import umap

In [ ]:
articles[76]

In [ ]:
featurize_stories(str(articles[12]), top_k = 3, max_len=512)

In [ ]:
rank_articles=[]
from tqdm import tqdm
for i in tqdm(range(len(articles))):
    try:
        cc = featurize_stories(str(articles[i]), top_k = 3, max_len=512)
        rank_articles.append([cc])
    except:
        pass

In [ ]:
flattened_list = [item for sublist in rank_articles for item in sublist]

In [ ]:
import pandas as pd
data=pd.DataFrame(flattened_list)  # each ranked feature is a row
data.drop_duplicates(inplace=True)

# g = graphistry.edges(data).materialize_nodes()
g = graphistry.nodes(data)
g2 = g.featurize()
# g2._clustersummary()

g3 = g.umap()
g3.dbscan()

In [ ]:
g3.encode_point_color('_dbscan',palette=["hotpink", "dodgerblue"],as_continuous=True).plot()


## funcoup scrape

In [ ]:
import requests, re
from bs4 import BeautifulSoup

# Send a GET request to the URL
response = requests.get('https://funcoup.org/downloads/')

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the links on the page
links = soup.find_all('a')
text = soup.get_text()

# Filter the links to only include those that end with .gz
gz_links = [link for link in links if link['href'].endswith('.gz')]
sizes = re.findall(r'(\d+\.\d+|\d+)(?=\s*(MB|KB))', text)

# For each .gz link, extract the file name and size
species=[]
for link, size in zip(gz_links, sizes):
    file_name = link.text
    file_size = size
    species.append([file_name, file_size])
species=pd.DataFrame(species, columns=['file_name', 'file_size'])

In [ ]:
def convert_to_kb(size):
    value, unit = size
    value = float(value)
    if unit == 'MB':
        value *= 1024  # convert MB to KB
    return value

# Add a new column 'type' based on the content of 'file_name'
species['type'] = species['file_name'].apply(lambda x: 'full' if 'full' in x else ('compact' if 'compact' in x else 'unknown'))

# Convert sizes to KB and sort
species['file_size_kb'] = species['file_size'].apply(convert_to_kb)
species = species.sort_values(by='file_size_kb', ascending=True)

# Drop the 'file_size_kb' column if you don't need it
species = species.drop(columns=['file_size_kb'])
species['name']=species['file_name'].str.split('_').str[1]

In [ ]:
# Convert the 'file_name' column to a dictionary
file_name_dict = species['name'].to_dict()

# Make a dictionary where the key and value are the same for each row
name_dict = {v: v for k, v in file_name_dict.items()}

In [ ]:
name_dict

In [ ]:
# Convert the dictionary to a list
name_list = list(name_dict.items())

In [ ]:
str(name_dict).replace('{', '(').replace('}', ')').replace('"', '')